System information (for reproducibility):

In [11]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [12]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2024spring/slides/15-linreg`


Status `~/Documents/github.com/ucla-biostat-257/2024spring/slides/15-linreg/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [7522ee7d] SweepOperator v0.3.4
  [37e2e46d] LinearAlgebra


## Comparing methods for linear regression

Methods for solving linear regression $\widehat \beta = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$:

| Method            | Flops                  | Remarks                 | Software | Stability   |
| :---------------: | :--------------------: | :---------------------: | :------: | :---------: |
| Sweep             | $np^2 + p^3$           | $(X^TX)^{-1}$ available | SAS      | less stable |
| Cholesky          | $np^2 + p^3/3$         |                         |          | less stable |
| QR by Householder | $2np^2 - (2/3)p^3$     |                         | R        | stable      |
| QR by MGS         | $2np^2$                | $Q_1$ available         |          | stable      | 
| QR by SVD         | $4n^2p + 8np^2 + 9p^3$ | $X = UDV^T$             |          | most stable |  

Remarks:

1. When $n \gg p$, sweep and Cholesky are twice faster than QR and need less space.  
2. Sweep and Cholesky are based on the **Gram matrix** $\mathbf{X}^T \mathbf{X}$, which can be dynamically updated with incoming data. They can handle huge $n$, moderate $p$ data sets that cannot fit into memory.  
3. QR methods are more stable and produce numerically more accurate solution.  
4. Although sweep is slower than Cholesky, it yields standard errors and so on.  
5. MGS appears slower than Householder, but it yields $\mathbf{Q}_1$.

> **There is simply no such thing as a universal 'gold standard' when it comes to algorithms.**

## Benchmark

In [13]:
using SweepOperator, BenchmarkTools, LinearAlgebra

linreg_cholesky(y::Vector, X::Matrix) = cholesky!(X'X) \ (X'y)

linreg_qr(y::Vector, X::Matrix) = X \ y

function linreg_sweep(y::Vector, X::Matrix)
    p = size(X, 2)
    xy = [X y]
    tableau = xy'xy
    sweep!(tableau, 1:p)
    return tableau[1:p, end]
end

function linreg_svd(y::Vector, X::Matrix)
    xsvd = svd(X)
    return xsvd.V * ((xsvd.U'y) ./ xsvd.S)
end

linreg_svd (generic function with 1 method)

In [14]:
using Random

Random.seed!(123) # seed

n, p = 10, 3
X = randn(n, p)
y = randn(n)

# check these methods give same answer
@show linreg_cholesky(y, X)
@show linreg_qr(y, X)
@show linreg_sweep(y, X)
@show linreg_svd(y, X);

linreg_cholesky(y, X) = [-0.07196570434574735, -0.13575699455859386, -0.18820199689456507]
linreg_qr(y, X) = [-0.07196570434574738, -0.1357569945585939, -0.18820199689456502]
linreg_sweep(y, X) = [-0.07196570434574734, -0.1357569945585939, -0.188201996894565]
linreg_svd(y, X) = [-0.07196570434574735, -0.13575699455859377, -0.1882019968945651]


In [15]:
n, p = 1000, 300
X = randn(n, p)
y = randn(n)

@benchmark linreg_cholesky(y, X)

BenchmarkTools.Trial: 6014 samples with 1 evaluation.
 Range (min … max):  679.875 μs …  12.927 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     742.875 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   830.349 μs ± 551.281 μs  ┊ GC (mean ± σ):  2.12% ± 7.10%

  ▃██▄▂▂▁▁                                                      ▁
  █████████▇▇▇▆▆▆▆▅▅▅▆▆▄▅▄▃▇▇█▇▅▆▅▆▅▆▅▆▅▅▅▄▅▅▁▅▄▁▅▄▄▅▃▁▄▃▄▅▄▁▃▄ █
  680 μs        Histogram: log(frequency) by time       2.51 ms <

 Memory estimate: 708.42 KiB, allocs estimate: 4.

In [16]:
@benchmark linreg_sweep(y, X)

BenchmarkTools.Trial: 1011 samples with 1 evaluation.
 Range (min … max):  4.634 ms …  17.202 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.758 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.947 ms ± 701.804 μs  ┊ GC (mean ± σ):  2.14% ± 6.21%

  ▄██▆▅▃▃▂▁                                                    
  ██████████▆▇▇▅▆▅▁▄▅▄▆▄▆▅▅▆▄▅▅▅▅▆▆▆▄▅▅▁▄▄▅▁▅▁▄▅▄▄▄▄▄▁▁▁▁▄▄▄▄ █
  4.63 ms      Histogram: log(frequency) by time      7.56 ms <

 Memory estimate: 2.99 MiB, allocs estimate: 6.

In [17]:
@benchmark linreg_qr(y, X)

BenchmarkTools.Trial: 317 samples with 1 evaluation.
 Range (min … max):  13.737 ms … 47.901 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.981 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.778 ms ±  3.004 ms  ┊ GC (mean ± σ):  0.91% ± 3.37%

   ▅▂█▅▃▂                                                      
  ███████▇▅▇▅▇▇▆▇▅▄▃▃▃▂▃▂▂▂▂▃▁▁▁▃▃▁▃▁▃▁▁▁▁▁▃▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂ ▃
  13.7 ms         Histogram: frequency by time        25.7 ms <

 Memory estimate: 2.53 MiB, allocs estimate: 1810.

In [18]:
@benchmark linreg_svd(y, X)

BenchmarkTools.Trial: 146 samples with 1 evaluation.
 Range (min … max):  28.634 ms … 61.672 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     32.921 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   34.305 ms ±  4.855 ms  ┊ GC (mean ± σ):  1.07% ± 2.36%

     ▅   ▃▃█ ▅▂   ▅                                            
  ▄▇▄█▆▆▇███▆██▆▅▆█▅▆▅▃▄▃▆▇▁▃▁▃▃▄▄▃▄▁▁▁▃▃▁▁▃▁▁▁▁▁▁▁▄▃▁▁▃▁▁▁▁▄ ▃
  28.6 ms         Histogram: frequency by time        50.2 ms <

 Memory estimate: 8.06 MiB, allocs estimate: 14.